# Problem 4

In [1]:
%pylab inline

import numpy as np
import pandas as pd
import itertools as it

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

Populating the interactive namespace from numpy and matplotlib


In [2]:
columns = ["edible", "cap-shape", "cap-surface", "cap-color", "bruises?",
            "odor", "gill-attachment", "gill-spacing", "gill-size", "gill-color",
            "stalk-shape", "stalk-root", "stalk-surface-above-ring",
            "stalk-surface-below-ring", "stalk-color-above-ring",
            "stalk-color-below-ring", "veil-type", "veil-color", "ring-number",
            "ring-type", "spore-print-color", "population", "habitat"
            ]
data = pd.read_csv("mushroom.data", names=columns, index_col=None)
data.head()

,edible,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,f,y,n,f,y,f,c,n,b,...,s,w,w,p,w,o,e,w,v,p
1,p,x,s,b,t,f,f,c,b,h,...,s,w,w,p,w,o,p,h,s,u
2,e,x,y,n,t,a,f,c,b,w,...,y,w,w,p,w,o,p,k,y,p
3,e,x,y,n,t,n,f,c,b,w,...,s,g,g,p,w,o,p,k,v,d
4,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,n,c,l


In [3]:
for column in columns:
    data[column] = pd.Categorical.from_array(data[column]).codes

data.head()

,edible,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,2,3,4,0,8,1,0,1,0,...,2,7,7,0,2,1,0,7,4,4
1,1,5,2,0,1,2,1,0,0,3,...,2,7,7,0,2,1,4,1,3,5
2,0,5,3,4,1,0,1,0,0,10,...,3,7,7,0,2,1,4,2,5,4
3,0,5,3,4,1,5,1,0,0,10,...,2,3,3,0,2,1,4,2,4,0
4,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,3,1,2


In [4]:
data['train'] = np.random.uniform(0, 1, len(data))

data_train = data[data['train'] <= 0.7]
data_test = data[data['train'] > 0.7]

y = data_train['edible']
X = data_train.drop('edible', axis = 1)

t = tree.DecisionTreeClassifier()
t = t.fit(X, y)

print "Train set score:", t.score(X, y)

y = data_test['edible']
X = data_test.drop('edible', axis = 1)

print "Test set score", t.score(X, y)

Train set score: 1.0
Test set score 1.0


In [5]:
dropped_data = pd.read_csv("mushroom.data", names=columns, index_col=None)
for column in columns:
    dropped_data[column] = pd.Categorical.from_array(dropped_data[column]).codes
dropped_data = dropped_data.drop([
        'ring-type', 'odor', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'gill-attachment', 'habitat', 'bruises?',
        'stalk-color-above-ring', 'stalk-color-below-ring', 'spore-print-color', 'cap-surface', 'veil-type', 'veil-color',
        'population', 'gill-spacing', 'stalk-shape'
    ], axis = 1)
dropped_data.head()

,edible,cap-shape,cap-color,gill-size,gill-color,stalk-root,ring-number
0,1,2,4,1,0,0,1
1,1,5,0,0,3,1,1
2,0,5,4,0,10,4,1
3,0,5,4,0,10,1,1
4,0,3,4,0,11,0,1


In [ ]:
classifiers = [tree.DecisionTreeClassifier, RandomForestClassifier, ExtraTreesClassifier]
params = [
    {
        "criterion": ["gini", "entropy"],
        "max_depth": [5, 10, None],
    },
    {
        "n_estimators": [5, 10],
        "criterion": ["gini", "entropy"],
        "max_depth": [5, 10, None],
    },
    {
        "n_estimators": [5, 10],
        "criterion": ["gini", "entropy"],
        "max_depth": [5, 10, None],
    },
]

for classifier_class, param in zip(classifiers, params):
    best = 0, {}
    for param_combination in [dict(zip(param, prod)) for prod in it.product(*(param[varName] for varName in param))]:
        classifier = classifier_class(**param_combination)
        repetitions = 100
        test_scores = 0
        for i in range(repetitions):
            dropped_data['train'] = np.random.uniform(0, 1, len(dropped_data))

            data_train = dropped_data[dropped_data['train'] <= 0.7]
            data_test = dropped_data[dropped_data['train'] > 0.7]

            y = data_train['edible']
            X = data_train.drop('edible', axis = 1)

            classifier = classifier.fit(X, y)

            y = data_test['edible']
            X = data_test.drop('edible', axis = 1)
            test_scores += classifier.score(X, y)
        
        avg_score = 100.0*test_scores/repetitions
        if avg_score > best[0]: best = avg_score, param_combination

    print "Best average score using {}:\n\tparams: {}\n\taverage score: {:.2f}%".format(classifier_class, best[1], best[0])

Best average score using <class 'sklearn.tree.tree.DecisionTreeClassifier'>:
	params: {'criterion': 'entropy', 'max_depth': None}
	average score: 96.03%
Best average score using <class 'sklearn.ensemble.forest.RandomForestClassifier'>:
	params: {'n_estimators': 10, 'criterion': 'entropy', 'max_depth': None}
	average score: 95.89%
